<a href="https://colab.research.google.com/github/mirjunaid26/datascientist/blob/main/ConvertingTabularDatasetToGraphDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Converting Tabular Dataset To Graph Dataset

I hope this notebook helps you to convert your CSV file into a graph dataset 🚀


`Step 0`

Bring some creativity and don't lose hope- It is very natural that it takes some time to rearrange the data in a graph format. Also, this notebook is only to help you to get started (you will have to transfer it to your specific use-case).

`Step 1`

To get started, identify the following things in your dataset (I have some real-world examples below in this notebook):

- Nodes (Items, People, Locations, Cars, ...)
- Edges (Connections, Interactions, Similarity, ...)
- Node Features (Attributes)
- Labels (Node-level, edge-level, graph-level)

and optionally:
- Edge weights (Strength of the connection, number of interactions, ...)
- Edge features (Additional (multi-dim) properties describing the edge)



`Step 2`

Do you have different node and edge types? (This means the nodes/edges have different attributes such as Cars vs. People)

- No, all my edges/nodes have the same type  --> **Proceed with 1.1**
- Yes, there are different relations and node types --> **Proceed with 1.2**


## 1.1 Homogeneous

`Step 3 / Example 1`

To make it as realistic as possible, I selected a random dataset I found on the internet that contains homogeneous nodes. This dataset is the [FIFA 21 Rating dataset](https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset), a dataset with soccer players.
Here we extract a small subset of the scraped data (there is much more available!) to build a graph dataset out of it. Have a look at the pandas Dataframe below.

In [ ]:
import pandas as pd

# Download data (quietly)
!wget -q https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_player.csv
!wget -q https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_player_skill.csv
!wget -q https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_team.csv

# Load data
player_df = pd.read_csv("tbl_player.csv")
skill_df = pd.read_csv("tbl_player_skill.csv")
team_df = pd.read_csv("tbl_team.csv")

# Extract subsets
player_df = player_df[["int_player_id", "str_player_name", "str_positions", "int_overall_rating", "int_team_id"]]
skill_df = skill_df[["int_player_id", "int_long_passing", "int_ball_control", "int_dribbling"]]
team_df = team_df[["int_team_id", "str_team_name", "int_overall"]]

# Merge data
player_df = player_df.merge(skill_df, on='int_player_id')
fifa_df = player_df.merge(team_df, on='int_team_id')

# Sort dataframe
fifa_df = fifa_df.sort_values(by="int_overall_rating", ascending=False)
print("Players: ", fifa_df.shape[0])
fifa_df.head()

Players:  18767


,int_player_id,str_player_name,str_positions,int_overall_rating,int_team_id,int_long_passing,int_ball_control,int_dribbling,str_team_name,int_overall
0,1,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,5.0,91,96,96,FC Barcelona,84
33,2,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,6.0,77,92,88,Juventus,83
57,3,Jan Oblak,GK,91,8.0,40,30,12,Atlético Madrid,83
121,5,Neymar da Silva Santos Júnior,"LW, CAM",91,7.0,81,95,95,Paris Saint-Germain,83
89,4,Kevin De Bruyne,"CAM, CM",91,2.0,93,92,88,Manchester City,85


Let's first identify the graph-specific things we need:

- `Nodes` - Football players (by ID)
- `Edges` - If they play for the same team (see explanation below)
- `Node Features` - The football player's position, specialities, ball control, ...
- `Labels` - The football player's overall rating (node-level regression task)


Nodes are usually very straight-forward to identify - here we even have IDs.
If you don't have a unique identifier, you need one, because you need to know between which nodes a connection exists!

 The most challenging task is typically to link these nodes somehow through edges. Here we define the edges based on the team assignment. With this dataset, we could predict the expected rating when a player switches to a new team or a new player is observed. Therefore we expect relational effects through the team assignment. Of course there are many other possibilities to define the edges such as:
- How many times two players played together (edge weight) --> Synergies
- How many times a player has won/los 1:1 duels (edge weight)
- Started their career in the same football club 
- Temporal edges: "Played together in the last 2 weeks"
- ...

As you can see, there are many choices how to combine instances in the dataframe. We will continue with the easiest approach, which is connecting them accoring to their team assignments.



In [ ]:
# Make sure that we have no duplicate nodes
max(fifa_df["int_player_id"].value_counts())

1

Each football player ID occurs only once in our dataset. 



> Note that we plan to build one single graph here! If individual node-id's occur more than once in your dataset, there are different options:

- You have multiple graphs that can contain the same node. In this case you need to iterate over each subset of your dataframe, that belongs to one individual graph and do the calculations on this subset
- You have to aggregate multiple rows into one. For example if you have transactional data (like a payment history), you would need to summarize this somehow into one feature vector, such as: #payments, payment amount, ...
- You have a temporal dataset and need to check section 2.)


`Preprocessing one single graph...`

`Step 4`: Extract the node features

The node features are typically represented in a matrix of the shape *(num_nodes, node_feature_dim)*.

For each of the football players, we simply extract their attributes. Because each player id is unique, we can easily do this based on the original dataframe. Have a look at the other examples in this notebook to see how an aggregation can look like if you have multiple rows for individual nodes.

In [ ]:
# Sort to define the order of nodes
sorted_df = fifa_df.sort_values(by="int_player_id")
# Select node features
node_features = sorted_df[["str_positions", "int_long_passing", "int_ball_control", "int_dribbling"]]
# Convert non-numeric columns
pd.set_option('mode.chained_assignment', None)
positions = node_features["str_positions"].str.split(",", expand=True)
node_features["first_position"] = positions[0]
# One-hot encoding
node_features = pd.concat([node_features, pd.get_dummies(node_features["first_position"])], axis=1, join='inner')
node_features.drop(["str_positions", "first_position"], axis=1, inplace=True)
node_features.head() 

,int_long_passing,int_ball_control,int_dribbling,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,91,96,96,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
33,77,92,88,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
57,40,30,12,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
89,93,92,88,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
121,81,95,95,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


That's already our node feature matrix. The number of nodes and the ordering is implicitly defined by it's shape. Each row corresponds to one node in our final graph. 

In [ ]:
# Convert to numpy
x = node_features.to_numpy()
x.shape # [num_nodes x num_features]

(18767, 18)

`Step 5`: Extract the labels

Those are simply the ratings of each of the players. This corresponds to a node-level prediction problem. Therefore we have as many labels as we have nodes. Of course it can happen that we don't have labels for all nodes and in this case it makes sense to define masks using Pytorch Geometric's helper functions.

In [ ]:
# Sort to define the order of nodes
sorted_df = fifa_df.sort_values(by="int_player_id")
# Select node features
labels = sorted_df[["int_overall"]]
labels.head()

,int_overall
0,84
33,83
57,83
89,85
121,83


In [ ]:
# Convert to numpy
y = labels.to_numpy()
y.shape # [num_nodes, 1] --> node regression

(18767, 1)

`Step 6`: Extract the edges

That's probably the trickiest part with a tabular dataset. You need to think of a reasonable way to connect your nodes. As mentioned previously, we will use the team assignment here.


> **AGAIN: There are many ways to connect the entities in a dataset and this approach is very trivial (as it will lead to disconnected subgraphs). If I wanted to build a real model from this dataset, I would probably look for a more sophisticated way to connect the players. Using a GNN is a bit overkill for the way I model the edges.**


We now need to find the pairs of players that are assigned to the same team.
Let's first check how many players per team we have.

In [ ]:
# Remap player IDs
fifa_df["int_player_id"] = fifa_df.reset_index().index

In [ ]:
# This tells us how many players per team we have to connect
fifa_df["str_team_name"].value_counts()

Everton                   36
Valencia CF               34
FC Nantes                 34
Villarreal CF             34
Real Valladolid CF        34
                          ..
Wellington Phoenix        19
Central Coast Mariners    19
Melbourne Victory         19
Brisbane Roar             19
Adelaide United           19
Name: str_team_name, Length: 681, dtype: int64

We now need to build all permutations of these players within one team, which corresponds to a fully-connected graph within each team-subgroup. We use the column int_player_id as indices for the edges. If there is for example a [0, 1] in the edge index, it means that the first and second node (regarding the previously defined node feature matrix) are connected.

In [ ]:
import itertools
import numpy as np

teams = fifa_df["str_team_name"].unique()
all_edges = np.array([], dtype=np.int32).reshape((0, 2))
for team in teams:
    team_df = fifa_df[fifa_df["str_team_name"] == team]
    players = team_df["int_player_id"].values
    # Build all combinations, as all players are connected
    permutations = list(itertools.combinations(players, 2))
    edges_source = [e[0] for e in permutations]
    edges_target = [e[1] for e in permutations]
    team_edges = np.column_stack([edges_source, edges_target])
    all_edges = np.vstack([all_edges, team_edges])
# Convert to Pytorch Geometric format
edge_index = all_edges.transpose()
edge_index # [2, num_edges]

array([[    0,     0,     0, ..., 18704, 18704, 18719],
       [    7,    32,    45, ..., 18719, 18751, 18751]])

The result are these source/target edge pairs. Here you can also model dircted or undirected edges by inluding both or just one direction (I included both). This COO format is usually chosen as it is more efficient than a *NxN* adjacency matrix.

`Step 7`: Build the dataset

Now we have all the components we need to build a graph for libraries like Pytorch Geometric or DGL. I won't install these libraries here, as this will make the notebook too bulky, but here are some code snippets for the final steps.


We need to pass the numpy arrays to the Data object, like this. If you have further attributes like edge_features, you can also pass them here.
```
from torch_geometric.data import Data
data = Data(x=x, edge_index=edge_index, y=y)
```

This data object represents one single graph.


Typically several graphs are combined in a dataset object. For this please refer to [the documentation](https://pytorch-geometric.readthedocs.io/en/latest/notes/create_dataset.html) or [this video](https://www.youtube.com/watch?v=QLIkOtKS4os&ab_channel=DeepFindr).
Other than that, you can also quickly build a dataloader as follows. Just create a list of all your graphs and pass them to the Pytorch Geometric dataloader.

```
from torch_geometric.loader import DataLoader
data_list = [Data(...), ..., Data(...)]
loader = DataLoader(data_list, batch_size=32)
```





In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=279639 sha256=05e61061318f5f28149cd48113c502a120db124fc34804d734361e2eaa7c99f9
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html
     |████████████████████████████████| 51 kB 139 kB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.14-cp37-cp37m-linux_x86_64.whl size=514667 sha256=22591bdc0c9290dea8b27a67f6596abfe298cbfd9b32d7f77fb9af4dd8c53e8a
  Stored in directory: /root/.cache/pip/wheels/3c/aa/62/db0259eae2abce84f1ee2cf1c531bba683aab4bf79054172f8
Successfully built torch-sparse
Look

In [ ]:
from torch_geometric.data import Data
data = Data(x=x, edge_index=edge_index, y=y)

In [ ]:
from torch_geometric.loader import DataLoader
data_list = [Data(...), ..., Data(...)]
loader = DataLoader(data_list, batch_size=32)

Next Step: Try GNN on this step...